In [1]:
# import python libraries
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
#download the files
train_df = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/train.csv")
test_df = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/test.csv")

In [3]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
# an example of a disaster tweet
train_df[train_df['target'] == 1]['text'].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# an example of a non disaster tweet
train_df[train_df['target'] == 0]['text'].values[0]

"What's up man?"

In [7]:
nltk.download('punkt')     ## required for tokenization
nltk.download('averaged_perceptron_tagger')     ## brequired for pos tagging
from nltk import word_tokenize, pos_tag

#tokenize the wprds
train_df_tokenize = [word_tokenize(word) for word in (train_df['text'])]
test_df_tokenize = [word_tokenize(word) for word in (test_df['text'])]
#train_df_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import re, string

# A function to remove hyperlinks, numbers and lemmatize the tokenized words
def clean_tweets(text, stop_words = ()):
    
    lemmatizer = WordNetLemmatizer()
    cleaned_tweet = []
    for word, tag in pos_tag(text):
    
        word = re.sub('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*','', word)
        word = re.sub("@(a-zA-Z0-9_)", '',word)
        word = re.sub('\w*\d\w*', '', word)
        word = re.sub('http|co|û_|amp', '', word)
    
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
            
        word = lemmatizer.lemmatize(word,pos)
        if len(word) > 0 and word not in string.punctuation and word.lower() not in stop_words:
            cleaned_tweet.append(word.lower())
            
    return cleaned_tweet      

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#create a list of the cleaned tokens
train_tokens = []
test_tokens = []

for token in train_df_tokenize:
    train_tokens.append(clean_tweets(token, stop_words))
    
for token in test_df_tokenize:
    test_tokens.append(clean_tweets(token, stop_words))
    
#train_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#A function to convert the token list to string
def list_to_string(token):
    space = " "
    s = []
    for text in token:
        s.append(space.join(text))
    return s

train_tokens_str = list_to_string(train_tokens)
test_tokens_str = list_to_string(test_tokens)

In [11]:

train_df['clean_text'] = train_tokens_str
test_df['clean_text'] = test_tokens_str
train_df.head()


,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents ask 'shelter place notify officers e...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo ruby alaska smoke wildfires pou...


In [14]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df['clean_text'][0:5])

print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 35)
[[0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]]


In [15]:
train_vectors = count_vectorizer.fit_transform(train_df["clean_text"])
test_vectors = count_vectorizer.transform(test_df["clean_text"])

In [16]:
clf = linear_model.RidgeClassifier()

scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=5, scoring="f1")
scores

array([0.58545455, 0.48759624, 0.53822153, 0.51838235, 0.62170543])

In [17]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [18]:
sample_submission = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/sample_submission.csv")

In [19]:
sample_submission["target"] = clf.predict(test_vectors)

In [20]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [21]:
sample_submission.to_csv("submission.csv", index=False)